In [1]:
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [2]:
test1_df = pd.read_excel("测试A.xlsx")
test2_df = pd.read_excel("测试B.xlsx")
test1_ANS = pd.read_csv('TESTA_ANS.csv', header=None)

In [35]:
test1_y = test1_ANS[test1_ANS.columns[-1]]

In [3]:
test1_id = test1_df['ID']
test2_id = test2_df['ID']
test1_df.drop(['ID'], axis=1, inplace=True)
test2_df.drop(['ID'], axis=1, inplace=True)

In [4]:
names = test1_df.columns
dts = test1_df.dtypes
drop_names = []
ratio = 0.1
for i in range(len(names)):
    n = names[i]
    tp = dts[i]
    trd = test2_df[n]
    ted = test1_df[n]
    if 'object' == str(tp):
        continue
    else:
        misstr = np.sum(trd.isnull())/500
        misste = np.sum(ted.isnull())/100
        if np.abs(misstr-misste) > ratio:
            drop_names.append(n)
        else:
            if 'int' in str(tp):
                continue
            trrd = trd[trd.notnull()]
            terd = ted[ted.notnull()]
            m1 = np.mean(trrd)
            s1 = np.std(trrd)
            m2 = np.mean(terd)
            s2 = np.std(terd)
            if np.abs(m1-m2)>ratio*np.abs(m1) and np.abs(s1-s2)>ratio*np.abs(s1):
                drop_names.append(n)
print(len(drop_names))

665


In [5]:
test1_df.drop(drop_names, axis=1, inplace=True)
test2_df.drop(drop_names, axis=1, inplace=True)

In [6]:
names = test1_df.columns
dfs = test1_df.dtypes

In [7]:
# ints or strings
ints = []
for i in range(len(names)):
    if 'int' in str(dfs[i]) or 'object' in str(dfs[i]):
        ints.append(names[i])
print(len(ints))

1630


In [9]:
df_X = pd.concat([test1_df, test2_df], axis=0)

In [10]:
remains1 = []
remains2 = []
tobedrop = []
for n in ints:
    lu1 = len(test1_df[n].unique())
    lu2 = len(df_X[n].unique())
    if lu1 < 10 and lu1 > 1:
        remains1.append(n)
    if lu2 < 10 and lu2 > 1:
        remains2.append(n)
    if lu1 != lu2:
        tobedrop.append(n)
print(len(remains1), len(remains2), len(tobedrop))

396 379 694


In [11]:
toberemain = list(set(remains1) & set(remains2) - set(tobedrop))
print(len(toberemain))

247


In [12]:
def dict2list(dic:dict):
    ''' 将字典转化为列表 '''
    keys = dic.keys()
    vals = dic.values()
    lst = [(key, val) for key, val in zip(keys, vals)]
    return lst

def LastRemainJudge(column):
    unique = set(column)
    uf = {}
    for u in unique:
        uf[u] = 0
    for c in column:
        uf[c] += 1
    sl = sorted(dict2list(uf), key = lambda x:x[1], reverse=True)
    x = sl[0][1]/len(column)
    sl = np.array(sl)
    if x < 0.8:
        #print(sl[:, 1]/len(column))
        return True
    else:
        return False

In [13]:
lm1 = []
lm2 = []
for n in toberemain:
    if LastRemainJudge(test1_df[n]):
        lm1.append(n)
    if LastRemainJudge(df_X[n]):
        lm2.append(n)
print(len(lm1), len(lm2), len(set(lm1)&set(lm2)))

182 185 180


In [14]:
names = df_X.columns
dts = df_X.dtypes
dfx_float_n = []
dfx_onehot_n = list(set(lm1) & set(lm2))
print(len(dfx_onehot_n))
for i in range(len(names)):
    n = names[i]
    d = dts[i]
    if 'float' in str(d):
        dfx_float_n.append(n)
print(len(dfx_float_n))
dfx_float = df_X[dfx_float_n]
dfx_onehot = df_X[dfx_onehot_n]
print(dfx_float.shape, dfx_onehot.shape)

180
5735
(221, 5735) (221, 180)


In [15]:
drop_names = []
names = dfx_float.columns
for n in names:
    x = np.sum(dfx_float[n].isnull())
    if x > 300:
        drop_names.append(n)
print(len(drop_names))

0


In [18]:
means = []
stds = []
drop_names = []
names = dfx_float.columns
for n in names:
    c = dfx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    means.append(tmpm)
    stds.append(tmps)
    if tmps == 0:
        drop_names.append(n)
print(len(set(means)), len(set(stds)))
print(len(drop_names))

2150 2154
0


In [17]:
dfx_float.drop(drop_names, axis=1, inplace=True)

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
# not move sames first, fill nan
added = set()
sames = []
for i in range(len(names)):
    tmpsl = []
    tmpsl.append(i)
    for j in range(i+1, len(names)):
        if means[i] == means[j] and stds[i] == stds[j]:
            if i not in added:
                added.add(i)
            if j not in added:
                added.add(j)
                tmpsl.append(j)
    if len(tmpsl) > 1:
        sames.append(tmpsl)
print(len(sames))

730


In [20]:
names = dfx_float.columns
drop_names = []
for i in sames:
    for j in i[1:]:
        drop_names.append(names[j])
print(len(drop_names))

3347


In [21]:
print(dfx_float.shape)

(221, 5503)


In [22]:
import copy
trainx_float = copy.copy(dfx_float)
trainx_float.drop(drop_names, axis=1, inplace=True)
print(trainx_float.shape)

(221, 2156)


In [23]:
names = trainx_float.columns
for n in names:
    c = trainx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    assert tmps>0
    trainx_float[n][c.isnull()] = tmpm
print(np.sum(trainx_float.isnull().values))

0


In [24]:
# it shall be that there is only thre lines with nan in log1p
mid = trainx_float-np.min(trainx_float)+1
trainx_log1p = np.log1p(mid)
del mid
print(np.sum(trainx_log1p.isnull().values))

0


In [25]:
for n in names:
    m1 = np.mean(trainx_float[n])
    s1 = np.std(trainx_float[n])
    assert s1 > 0
    trainx_float[n] = (trainx_float[n]-m1)/s1
    m2 = np.mean(trainx_log1p[n])
    s2 = np.std(trainx_log1p[n])
    assert s2 > 0
    trainx_log1p[n] = (trainx_log1p[n]-m2)/s2

In [26]:
# make onehots onehot
ohnames = dfx_onehot.columns
for i in range(len(ohnames)):
    n = ohnames[i]
    tmp = pd.get_dummies(dfx_onehot[n], prefix='onehot_'+str(i))
    dfx_onehot = pd.concat([dfx_onehot, tmp], axis=1)
    print(dfx_onehot.shape)

(221, 185)
(221, 188)
(221, 194)
(221, 197)
(221, 201)
(221, 208)
(221, 210)
(221, 215)
(221, 218)
(221, 220)
(221, 223)
(221, 225)
(221, 227)
(221, 233)
(221, 236)
(221, 239)
(221, 243)
(221, 249)
(221, 255)
(221, 259)
(221, 261)
(221, 269)
(221, 273)
(221, 276)
(221, 279)
(221, 286)
(221, 288)
(221, 296)
(221, 299)
(221, 302)
(221, 305)
(221, 310)
(221, 312)
(221, 314)
(221, 322)
(221, 324)
(221, 333)
(221, 340)
(221, 346)
(221, 348)
(221, 352)
(221, 355)
(221, 357)
(221, 364)
(221, 373)
(221, 378)
(221, 382)
(221, 387)
(221, 392)
(221, 400)
(221, 402)
(221, 405)
(221, 408)
(221, 410)
(221, 413)
(221, 420)
(221, 424)
(221, 429)
(221, 432)
(221, 434)
(221, 436)
(221, 438)
(221, 440)
(221, 444)
(221, 447)
(221, 450)
(221, 454)
(221, 457)
(221, 462)
(221, 467)
(221, 471)
(221, 473)
(221, 476)
(221, 478)
(221, 481)
(221, 484)
(221, 486)
(221, 492)
(221, 498)
(221, 500)
(221, 504)
(221, 507)
(221, 510)
(221, 513)
(221, 515)
(221, 517)
(221, 522)
(221, 526)
(221, 531)
(221, 539)
(221, 541)

In [27]:
dfx_onehot.drop(ohnames, axis=1, inplace=True)
print(dfx_onehot.shape)

(221, 699)


In [28]:
dfx_float = trainx_float
dfx_log1p = trainx_log1p

In [29]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [30]:
def MSE(y_raw, y_pred):
    print(np.mean(np.square(y_raw-y_pred)))

In [31]:
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV 

/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [32]:
from sklearn.model_selection import KFold 
kf = KFold(n_splits=5, random_state=2018)
def get_oof(rgr, xtrain, ytrain, xtest, lenot):
    oof_train = np.zeros((lenot,))
    oof_test = np.zeros((100,))
    oof_test_skf = np.empty((5, 100))
    for i, (train_index, test_index) in enumerate(kf.split(xtrain)):
        kf_xtrain = xtrain[train_index]
        kf_ytrain = ytrain[train_index]
        kf_xtest = xtrain[test_index]
        rgr.fit(kf_xtrain, kf_ytrain)
        oof_train[test_index] = rgr.predict(kf_xtest)
        oof_test_skf[i, : ] = rgr.predict(xtest)
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1,1), oof_test.reshape(-1,1)

In [33]:
dfx = pd.concat([dfx_log1p, dfx_onehot], axis=1)
dfx.shape

(221, 2855)

In [37]:
subxtrain, subxval, subytrain, subyval = train_test_split(dfx[:100].values, test1_y.values, 
                                                          test_size=0.2, random_state=20)

xgb = XGBRegressor(gamma=0.0004, max_depth=3, objective='reg:gamma', subsample=0.8)
xgb.fit(subxtrain, test1_y)
tr_pred = xgb.predict(dfx[:100])
te_pred = xgb.predict(dfx[500:600])
MSE(tr_pred, test1_y.values)
MSE(te_pred, test1_y.values)

0.000408923291347
